In [1]:
import pandas as pd
import pickle

In [2]:
with open('./data/tokenizedTextNoun','rb') as f:
    docs = pickle.load(f)

In [3]:
from gensim import corpora

def create_corpus(docs):
    dictionary = corpora.Dictionary(docs)
    print('Number of unique words in initital documents:', len(dictionary))

    dictionary.filter_extremes(no_below=50, no_above=0.75)
    print('Number of unique words after removing rare and common words:', len(dictionary))
    
    ## term document frequency
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    return corpus,dictionary

In [4]:
%time corpus,dictionary = create_corpus(docs)

Number of unique words in initital documents: 86955
Number of unique words after removing rare and common words: 10053
CPU times: user 22.1 s, sys: 309 ms, total: 22.4 s
Wall time: 22.5 s


In [6]:
dictionary.save('./ModelNoun19/dictionary.dic')

In [7]:
#Human readable format of corpus (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[1000:1002]]

[[('body', 3),
  ('friend', 1),
  ('less', 1),
  ('long', 1),
  ('time', 11),
  ('watch', 1),
  ('week', 2),
  ('work', 2),
  ('different', 1),
  ('finish', 1),
  ('amount', 2),
  ('check', 1),
  ('much', 3),
  ('change', 1),
  ('day', 1),
  ('diet', 1),
  ('easy', 1),
  ('exercise', 10),
  ('family', 1),
  ('favorite', 2),
  ('goal', 2),
  ('good', 2),
  ('help', 1),
  ('journal', 1),
  ('key', 2),
  ('lifestyle', 1),
  ('list', 3),
  ('minute', 1),
  ('something', 2),
  ('step', 1),
  ('way', 1),
  ('phone', 1),
  ('right', 1),
  ('building', 1),
  ('measure', 1),
  ('note', 1),
  ('size', 1),
  ('strength', 1),
  ('water', 5),
  ('avoid', 1),
  ('common', 1),
  ('lot', 1),
  ('arm', 1),
  ('inch', 1),
  ('tape', 1),
  ('content', 1),
  ('helpful', 1),
  ('soft', 1),
  ('start', 1),
  ('stick', 1),
  ('sugar', 1),
  ('computer', 1),
  ('ipod', 1),
  ('lose', 3),
  ('new', 1),
  ('activity', 2),
  ('enough', 2),
  ('member', 1),
  ('tennis', 1),
  ('whatev', 1),
  ('enjoyable', 1),
  

In [8]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 19
chunksize = 500 # size of the doc looked at every pass
passes = 100 # number of passes through documents
iterations = 100
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
#temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time ldamodel = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every,per_word_topics=True,random_state=46)

CPU times: user 1h 53min 31s, sys: 39.5 s, total: 1h 54min 10s
Wall time: 1h 18min 34s


# save the model

In [9]:
ldamodel.save('./ModelNoun19/ldamodel.model')

In [10]:
# Print the Keyword in the 10 topics
from pprint import pprint
#pprint(ldamodel.print_topics())

In [11]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [12]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/anaconda3/envs/py36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.075174 -0.124553       1        1  15.497785
9      0.022129 -0.180575       2        1   7.213546
4      0.098115  0.196324       3        1   7.132782
6     -0.178713 -0.107577       4        1   6.881946
2      0.106300  0.035304       5        1   6.567375
17     0.133285 -0.040165       6        1   5.955286
18     0.229244  0.001163       7        1   5.798595
7     -0.124579 -0.052581       8        1   5.349851
1      0.000607 -0.184610       9        1   4.879617
15     0.137783 -0.105104      10        1   4.871169
10     0.236000  0.124833      11        1   4.768212
13     0.173407  0.112271      12        1   4.708916
3     -0.150596  0.140766      13        1   3.845067
5     -0.194997  0.224296      14        1   3.580416
12    -0.165125  0.053551      15        1   3.139754
14    -0.016091 -0.084908      16        1   3.057596
11    -0.096116 -0.135528      17        1   2.604202
16     0.016973 -0.027418      18        1   2.148652
0     -0.152454  0.154512      19        1   1.999232, topic_info=     Category           Freq         Term          Total  loglift  logprob
752   Default   48866.000000          dog   48866.000000  30.0000  30.0000
101   Default   51109.000000        child   51109.000000  29.0000  29.0000
432   Default   74674.000000        water   74674.000000  28.0000  28.0000
460   Default   46260.000000         hair   46260.000000  27.0000  27.0000
188   Default   43406.000000         food   43406.000000  26.0000  26.0000
73    Default   35480.000000       number   35480.000000  25.0000  25.0000
4714  Default   31014.000000          cat   31014.000000  24.0000  24.0000
177   Default   59122.000000          day   59122.000000  23.0000  23.0000
289   Default   32424.000000        color   32424.000000  22.0000  22.0000
216   Default   66099.000000       people   66099.000000  21.0000  21.0000
1249  Default   20941.000000         word   20941.000000  20.0000  20.0000
1358  Default   20501.000000         game   20501.000000  19.0000  19.0000
44    Default  128810.000000         time  128810.000000  18.0000  18.0000
14    Default   42354.000000       friend   42354.000000  17.0000  17.0000
129   Default   29819.000000         name   29819.000000  16.0000  16.0000
257   Default   19475.000000       screen   19475.000000  15.0000  15.0000
38    Default   33078.000000         skin   33078.000000  14.0000  14.0000
661   Default   20282.000000         file   20282.000000  13.0000  13.0000
1018  Default   19820.000000       doctor   19820.000000  12.0000  12.0000
582   Default   39432.000000          top   39432.000000  11.0000  11.0000
531   Default   42276.000000         hand   42276.000000  10.0000  10.0000
37    Default   44185.000000         side   44185.000000   9.0000   9.0000
288   Default   26742.000000        clean   26742.000000   8.0000   8.0000
470   Default   26981.000000          oil   26981.000000   7.0000   7.0000
207   Default   34799.000000       minute   34799.000000   6.0000   6.0000
137   Default   41853.000000       person   41853.000000   5.0000   5.0000
13    Default   25566.000000          dry   25566.000000   4.0000   4.0000
930   Default   15485.000000         baby   15485.000000   3.0000   3.0000
133   Default   15622.000000       parent   15622.000000   2.0000   2.0000
369   Default   26783.000000        piece   26783.000000   1.0000   1.0000
...       ...            ...          ...            ...      ...      ...
5323  Topic19     779.333862          joy     780.336792   3.9111  -5.8686
3758  Topic19     778.612366       theory     779.615295   3.9111  -5.8695
5050  Topic19     742.730530  achievement     743.733459   3.9111  -5.9167
5996  Topic19     721.624146     approval     722.627075   3.9110  -5.9455
2867  Topic19    3544.552002        theme    3754.634277   3.8548  -4.3539
666   Topic19    9480.372070        music   1097

### Model Coherency

In [13]:
import time
from gensim.models import CoherenceModel

start=time.time()

coherence_model_lda = CoherenceModel(model=ldamodel, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

end=time.time()
print(end-start)


Coherence Score:  0.6145618527009855
279.1745400428772


In [14]:
### Model Perplexity

In [15]:
%time print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.


Perplexity:  -8.468058716249068
CPU times: user 1min 33s, sys: 3.4 s, total: 1min 36s
Wall time: 1min 13s
